#LDA (비지도 학습)

In [1]:
# 토픽 모델링, 문서의 단어 분포 패턴에서 토픽을 추출하는 알고리즘
# LDA는 문서가 토픽의 혼합으로 구성되어 있으면, 토픽은 각각의 단어 분포를 가지고 있다고 가정
# 각 단어에 어떤 토픽에서 생성되었을지를 추첮앟여 문서의 주제를 파악

In [2]:
# 1. 토픽의 갯수를 지정 (전체 글에서 비슷한 글끼리 묶어줌)
# 2. 토픽의 혼합 비율을 무작위로 설정
# 3. 모든 단어를 특정 토픽에 할당
# 4. 아래 작업 반복
#    1) 현재 문서의 모든 단어에 대해 해당 단어가 속한 토픽 할당
#    2) 현재 문서에서 각 토픽이 할당된 비율 업데이트
#    3) 모든 문서에서 해당 토픽이 할당된 비율 업데이트
# 각 문서에서 토픽이 할당된 비율을 이용하여 해당 문서의 주제를 추정

In [3]:
# 댓글 분석 or SNS 상의 데이터 분석

In [4]:
!pip install konlpy

In [5]:
# LDA 시각화 패키지
!pip install -U pyLDAvis

### 데이터 로드

In [14]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/메타버스_아카데미_2기/딥러닝/7월/data/appreply.csv')
df

,Unnamed: 0,text,score
0,0,NaN,4
1,1,NaN,5
2,2,NaN,1
3,3,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
4,4,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
...,...,...,...
998,998,갑자기 로그아웃 되더니 비밀번호 변경 실패 메세지가 계속 뜨네요. 휴대폰 번호로 인...,1
999,999,기사님이 상품 픽업을 하셨는지 표시되면 더 좋을 것 같습니다. 가게에서 조리가 늦게...,3
1000,1000,요즘 요기요 보다 배민을 많이 쓰는 사람입니다 전화보다 앱을 써서 좀더 간편하고 다...,3
1001,1001,취소 됐으면 적어도 전화 주는 제도는 있어야하는거 아닌가요? 주문해놓고 다른거 하는...,1


In [15]:
df = df.iloc[3:]
df = df.reset_index()
df = df.drop(columns=['index','Unnamed: 0'])
df.head()

,text,score
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3


### 형태소 추출

In [16]:
import konlpy
import re
from tqdm import tqdm

In [24]:
def tokenize_text(text):
  text = re.sub(r'[^ㄱ - | 가-힣\s]','',text) # \s : 공백

  okt = konlpy.tag.Okt()
  okt_morphs = okt.pos(text)

  words = []

  for word,pos in okt_morphs:

    # 명사 동사 형용사만
    if pos in ['Adjective','Verb','Noun']:

      # LDA는 문서형태로 넣어야함
      words.append(word)

  word_str = ' '.join(words)
  return word_str

tokenize_text("배달의 민족 주문시 리뷰를 자주 참고 하는 편입니다.")

'배달 민족 주문 시 리뷰 자주 참고 하는 편입 니'

In [25]:
token_list = []
for text in tqdm(df['text']):
  token_list.append(tokenize_text(text))

100%|██████████| 1000/1000 [00:21<00:00, 45.87it/s]


In [26]:
token_list[0]

'배달 민족 주문 시 리뷰 자주 참고 하는 편입 니 한가지 건의 사항 있다면 최신 점순 뿐 아니라 제 주문 하고 자하 메뉴 특정해서 그 메뉴 리뷰 확인 할 수 있는 기능 있으면 좋을 것 같습니다 메뉴 검색 기능 리뷰 특정 메뉴 검색 기능 필요합니다 주문 수가 많지 않은 메뉴 리뷰 보기 위해 드 래그 하느라 시간 소요 되는 비 효율 발생 합니다 긍정 검토 해 주심 좋을 것 같습니다'

In [28]:
corpus_list = []
for idx in range(len(token_list)):
  corpus = token_list[idx]
  # 짧은 문장들만 저장
  # 짧은 문장안의 몇개안되는 단어들로만 연산하기 때문에 오분류할 수 있음
  if len(set(corpus.split()))<3:
    corpus_list.append(corpus)
corpus_list

[]

In [30]:
# 만약 짧은 문장이 corpus_list에 있다면 제거
for corpus in corpus_list:
  token_list.remove(corpus)

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [33]:
# LDA는 count 기반의 vectorizer만 사용!



# max_df : 0.1, 전체 문서에서 10%이상 나오는 단어 무시
# min_df : 2, 2개 미만 문서에서 나오면 무시
# ngram_range = (1,2), 1개 or 2개로 이루어진 단어
# https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer
count_vectorizer = CountVectorizer(max_df=0.1,min_df=2,max_features=1000,ngram_range=(1,2))

feat_vect = count_vectorizer.fit_transform(token_list)
feat_vect.shape

(1000, 1000)

In [46]:
feat_vect.toarray()[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [48]:
count_vectorizer.vocabulary_

{'민족': 303,
 '자주': 714,
 '참고': 815,
 '한가지': 934,
 '건의': 45,
 '사항': 424,
 '있다면': 691,
 '최신': 832,
 '아니라': 511,
 '하고': 897,
 '확인': 990,
 '있으면': 705,
 '좋을': 774,
 '같습니다': 25,
 '특정': 860,
 '수가': 462,
 '않은': 543,
 '보기': 366,
 '위해': 625,
 '되는': 166,
 '발생': 323,
 '합니다': 947,
 '배달 민족': 330,
 '건의 사항': 46,
 '메뉴 리뷰': 269,
 '있는 기능': 686,
 '기능 있으면': 87,
 '좋을 같습니다': 775,
 '메뉴 검색': 267,
 '검색 기능': 49,
 '특정 메뉴': 861,
 '리뷰 보기': 222,
 '했던': 976,
 '목록': 290,
 '있었으면': 702,
 '좋겠어요': 767,
 '분명': 387,
 '시킨': 485,
 '기억': 92,
 '있는데': 687,
 '찾기': 816,
 '좋을거': 776,
 '같아요': 28,
 '주문 했던': 790,
 '기능 있었으면': 86,
 '있었으면 좋겠어요': 704,
 '화면': 989,
 '전체': 732,
 '포장': 880,
 '크롤': 852,
 '아래': 519,
 '하는데': 913,
 '넘어가서': 112,
 '되는데': 170,
 '정말': 746,
 '불편합니다': 399,
 '마트': 229,
 '하나': 900,
 '선택': 450,
 '했으면': 981,
 '좌우': 778,
 '안되나요': 527,
 '가끔': 8,
 '아니고': 510,
 '불편하고': 395,
 '씁니다': 508,
 '배달 포장': 343,
 '낮은': 106,
 '정렬': 744,
 '가장': 16,
 '위로': 622,
 '지역': 803,
 '따라': 200,
 '추가': 834,
 '있다고': 690,
 '별도': 362,
 '체크': 824,
 '하게': 

In [34]:
lda = LatentDirichletAllocation(n_components=5)
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=5)

In [36]:
feature_names = count_vectorizer.get_feature_names_out()
feature_names

array(['가게 검색', '가게 메뉴', '가게 목록', '가게 배달', '가게 주문', '가격', '가기', '가까운',
       '가끔', '가능', '가능하게', '가능한', '가맹', '가서', '가요', '가입', '가장', '가족',
       '가지', '갈수록', '감사합니다', '갑자기', '강제', '같고', '같네요', '같습니다', '같아',
       '같아서', '같아요', '같은', '같은 경우', '같은거', '같은데', '같음', '개발', '개발자', '개인',
       '개편', '갤럭시', '거기', '거나', '거리', '거의', '건가', '건물', '건의', '건의 사항',
       '건지', '걸리고', '검색 기능', '겁니다', '결과', '결재', '결제 수단', '경우', '경험', '계속',
       '계정', '고객', '고객 센터', '고려', '고민', '고쳐주세요', '공지', '과정', '관련', '관리',
       '광고', '굉장히', '구매', '구분', '구성', '굳이', '그거', '그것', '그게', '그냥', '그대로',
       '그런', '그럴', '그럼', '근처', '글자', '금액', '기간', '기기', '기능 있었으면',
       '기능 있으면', '기능 추가', '기본', '기분', '기사', '기억', '기업', '기요', '기입', '기존',
       '기준', '까요', '깔고', '깜빡', '나서', '나오고', '나오는', '나중', '남깁니다', '낮은',
       '낮은 정렬', '내고', '내려서', '내용', '넘게', '넘어가서', '네이버', '네이버 로그인', '네트워크',
       '노력', '노출', '높은', '누가', '누구', '누락', '누르고', '누르면', '눌러도', '눌러서',
       '느낌', '는걸', '늦게', '다르게', '다른 배달', '다른 어플', '다만', '다시 설치',

In [37]:
# 토픽을 5개로 설정함
# 높은 수가 가중치가 높은 단어를 뜻함
lda.components_

array([[ 0.205572  ,  0.20057274,  0.20159621, ..., 14.69658203,
         4.22970067,  0.20133208],
       [ 0.2009562 ,  1.6964736 ,  0.20007533, ...,  2.44469173,
         0.20000157,  0.20155973],
       [ 2.65339853,  0.20389043,  0.20000612, ...,  6.45553726,
         4.1700886 ,  0.2006745 ],
       [ 0.20639825,  0.20270056,  8.19617914, ...,  0.20088299,
         0.20004975,  0.20362085],
       [ 7.73367502,  7.69636267,  0.20214321, ...,  0.20230599,
         0.20015942,  7.19281285]])

In [44]:
len(lda.components_[0])

1000

### LDA 분류한 토픽별 가중치가 높은 단어들 추출

In [42]:
def display_topics(model,feature_names,num_top_words=10):
  for topic_index,topic in enumerate(model.components_):
    print("Topic : ",topic_index)
    topic_word_indexes = topic.argsort()[::-1]
    top_indexes = topic_word_indexes[:num_top_words]
    print("top_indexes : ",top_indexes)
    feature_concat = ''.join([feature_names[i] for i in top_indexes])
    print("feature_concat : ",feature_concat)

display_topics(lda,feature_names,20)

Topic :  0
top_indexes :  [213 600  56 678 990 298 793 841 734 954 452 608 660 989 358 547 950 296
 198 426]
feature_concat :  로그인오류계속입력확인문제주소카드전화해도설정완료인증화면번호알림해결문의등록삭제
Topic :  1
top_indexes :  [838  58 734 809 430 447 455 475 897 989 310 477  59 969 588 932  83 648
 298 947]
feature_concat :  취소고객전화진짜상담서비스센터시스템하고화면바로시켜고객 센터했는데연결하지금액이용문제합니다
Topic :  2
top_indexes :  [945  15 426 660 297 237 990 375 946 853 969 994 213 453 412  56 855 154
 596  83]
feature_concat :  할인가입삭제인증문자만원확인본인할인 쿠폰클릭했는데회원로그인설치사람계속탈퇴도착예정금액
Topic :  3
top_indexes :  [290 880  67 989 441 802 900 382 644  29 809 648 744  76 823 366 106 961
  41 400]
feature_concat :  목록포장광고화면생각지도하나부분이벤트같은진짜이용정렬그냥천원보기낮은해주세요거리불편해요
Topic :  4
top_indexes :  [767 766 834 694  91 450 424 702 705  54 330 303 698 614 746 328  28 412
 648 947]
feature_concat :  좋겠어요좋겠습니다추가있습니다기사선택사항있었으면있으면경우배달 민족민족있어서요청정말배달 기사같아요사람이용합니다


###LDA 시각화

In [49]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda,feat_vect,count_vectorizer)
pyLDAvis.display(vis)

### 각 문서별 높은 확률의 토픽

In [50]:
sent_topic = lda.transform(feat_vect)

# 0번 문장에 대한 토픽 확률
# 총합이 1 (softmax)

sent_topic[0]

array([0.00519094, 0.00517947, 0.00518226, 0.0051851 , 0.97926222])

In [51]:
len(sent_topic)

1000

In [52]:
sent_topic.shape

(1000, 5)

In [53]:
sent_topic_per_list = []
for n in range(sent_topic.shape[0]):
  sent_topic_list = sent_topic[n].argmax()
  topic_per = sent_topic[n].max()
                            # 수, 토픽분류     , 가장 높은 토픽일 확률
  sent_topic_per_list.append([n,sent_topic_list,topic_per])

In [54]:
topic_per_df = pd.DataFrame(sent_topic_per_list,columns=['no','토픽번호','확률'])
topic_per_df

,no,토픽번호,확률
0,0,4,0.979262
1,1,4,0.912826
2,2,3,0.973061
3,3,3,0.973814
4,4,0,0.877253
...,...,...,...
995,995,0,0.539635
996,996,4,0.977973
997,997,2,0.789349
998,998,1,0.905355


### 데이터프레임 결합

In [55]:
doc_topic_df = topic_per_df.join(df)
doc_topic_df

,no,토픽번호,확률,text,score
0,0,4,0.979262,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
1,1,4,0.912826,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
2,2,3,0.973061,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1
3,3,3,0.973814,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2
4,4,0,0.877253,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3
...,...,...,...,...,...
995,995,0,0.539635,갑자기 로그아웃 되더니 비밀번호 변경 실패 메세지가 계속 뜨네요. 휴대폰 번호로 인...,1
996,996,4,0.977973,기사님이 상품 픽업을 하셨는지 표시되면 더 좋을 것 같습니다. 가게에서 조리가 늦게...,3
997,997,2,0.789349,요즘 요기요 보다 배민을 많이 쓰는 사람입니다 전화보다 앱을 써서 좀더 간편하고 다...,3
998,998,1,0.905355,취소 됐으면 적어도 전화 주는 제도는 있어야하는거 아닌가요? 주문해놓고 다른거 하는...,1


In [56]:
doc_topic_df['토픽번호'].value_counts()

토픽번호
4    237
0    224
1    220
3    208
2    111
Name: count, dtype: int64

### 토픽별 확률이 높은 데이터를 찾아 어떤 토픽인지 라벨링

In [59]:
from pandas.core.algorithms import doc
# topic = 0,1,2,3,4
for topic in range(len(doc_topic_df['토픽번호'].unique())):
  print("Top No : ",topic)
  top_per_topics = doc_topic_df[doc_topic_df['토픽번호']==topic].sort_values(by='확률',ascending=False)
  print(top_per_topics['text'].iloc[0])
  print(top_per_topics['text'].iloc[1])
  print(top_per_topics['text'].iloc[2])

Top No :  0
아이디 찾기를 하는데 휴대폰 인증을 하면 다시 로그인 하기가 뜹니다. 로그인 버튼을 누르면 당연하게도 그냥 로그인 창으로 연결되고요. 아이디를 찾아야 로그인을 하잖아요.... +) 몇번 시도를 하니 아이디를 일부 알려주는 화면이 연결됐습니다. 그런데 입력시에 오타가 났는지 제가 쓰는 이메일 주소가 아니었어요. 모르는 아이디에 비밀번호라 당연히 로그인 정보가 틀렸는데 수십번을 해도 로그인 시도 카운팅이 제대로 되지 않네요. 계정정보가 일치하지 않습니다 (4회 남음) 에서 변하지가 않아요. 일정 횟수 로그인 시도가 이루어지면 보통은 휴대폰이나 메일 등으로 계정 정보와 함께 로그인 시도 알림이 가서 계정을 잠그거나 로그인 정보 자체를 바꾸거나 탈퇴할 수 있게 해주던데 그런 장치는 없는 건가요? 아이디 자체를 모르니 탈퇴 후 재가입도 불가능하고, 앱 내의 고객센터에도 관련 카테고리는 없고.. 휴대폰 번호를 새로 발급 받지 않고는 비회원으로만 주문하게 됐는데 상당히 불편합니다..
아니 지금 계속 네트워크 연결이 불안정 하다고 메시지 뜨면서 아무것도 안돼서 음식 주문을 못하고 있는데 왜 이런건가요?... 보니까 저 말고도 이런증상 있으신 분들 꾀 있으신거 같고 고객센터 문의 한지도 일주일 가까이 되어 가는데 연락도 없고 앱은 안되고...?? 스마트폰 재부팅, 앱 캐시삭제, 앱 재설치 등등 할수 있는건 다 해보고 문의 한겁니다. 여태 잘 되다가 안되는거 보면 앱 문제거나 업뎃하고 다른 앱이랑 충돌 나서 그러는거 같은데 전화주문 다 없에고 업계 독점했으면 이렇게 무책임 하게 운영해도 되는 건가요?? 이럴거면 사업 접고 예전 전화주문 시대로 돌아가시죠?? 언론에 제보하던가 해야지 원... 지들이 뭐 잘못 만져서 소비자 불편겪게 해놓고 재설치 캐시삭제 다 해봤다니까 로봇마냥 맨날 복붙 똑같은 답변 레전드네 언론사에 제보 할게요~ 수고하세요~
배달 주소 수정이 안됨 이사를 하여 다른 주소로 수정을 하려 하였으나 앱 내에서 환경설정에도 수정이 불가능 배달 중 